In [18]:
import os
import h5py
import graphics
import glob

import meshplot as mp
import numpy as np

In [19]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [20]:
from skimage.measure import marching_cubes

In [21]:
import sys
sys.path.append('..')

from utils.loading import load_config_from_json
from training.utils import get_dataset

In [22]:
ROOT_DIR = "../experiments"

In [23]:
@interact
def select_experiment(experiment=os.listdir(ROOT_DIR)):
    global selected_experiment
    global experiment_path
    
    selected_experiment = experiment
    experiment_path = os.path.join(ROOT_DIR, experiment)

interactive(children=(Dropdown(description='experiment', options=('shapenet_noise_005', 'modelnet_outliers_99'…

In [29]:
@interact
def select_version(version=os.listdir(experiment_path)):
    global selected_version
    global version_path
    
    selected_version = version
    version_path = os.path.join(experiment_path, version)

interactive(children=(Dropdown(description='version', options=('version_0', '0', 'version_1', 'version_2', '2'…

In [30]:
@interact
def select_stage(stage=os.listdir(os.path.join(version_path, 'scenes'))):
    
    global selected_stage 
    global stage_path
    
    selected_stage = stage
    stage_path = os.path.join(version_path, 'scenes', stage)

interactive(children=(Dropdown(description='stage', options=('last', 'best'), value='last'), Output()), _dom_c…

In [31]:
@interact
def select_mode(mode=os.listdir(stage_path)):
    global selected_mode
    global result_path
    
    selected_mode = mode
    result_path = os.path.join(stage_path, mode)

interactive(children=(Dropdown(description='mode', options=('val', 'train', 'test'), value='val'), Output()), …

In [32]:
def extract_scenes(result_path):
    scenes = glob.glob(os.path.join(result_path, '*.tsdf.hf5'))
    scenes = [s.split('/')[-1].replace('.tsdf.hf5', '') for s in scenes]
    return scenes

def load_grid(path):
    
    with h5py.File(path + '.tsdf.hf5', 'r') as file:
        tsdf = file['TSDF'][:][0]
    return tsdf

def load_dataset(path, mode, data_path=None):
    config = load_config_from_json(path)
    
    if data_path is not None:
        config.SETTINGS.data_path = data_path
    config.DATA.train_list = '../' + config.DATA.train_list
    config.DATA.val_list = '../' + config.DATA.val_list
    config.DATA.test_list = '../' + config.DATA.test_list

    dataset = get_dataset(config, mode)
    return dataset

def load_groundtruth(dataset, scene):
    grid = dataset.get_grid(scene)
    grid.transform("normal")
    grid.volume *= grid.resolution
    return grid.volume

scenes = extract_scenes(result_path)
# dataset = load_dataset(version_path, 'test', data_path='/media/weders/HV620S/data/modelnet/processed')

In [33]:
@interact
def visualize_scene(scene=scenes):
    prediction = load_grid(os.path.join(result_path, scene))
    # groundtruth = load_groundtruth(dataset, scene.replace('.', '/'))
    print("loaded")
    verts_est, faces_est, _, _ = marching_cubes(prediction, level=0.)
    #verts_gt, faces_gt, _, _ = marching_cubes(groundtruth, level=0.)
    
    p = mp.subplot(verts_est, faces_est, s=[1, 2, 0])
    # p = mp.subplot(verts_gt, faces_gt, s=[1, 2, 1], data=p)
    
    occupied_est = prediction < 0.
    # occupied_gt = (groundtruth < 0.) & (groundtruth > -0.02)
    
    # intersection = occupied_est & occupied_gt
    # union = occupied_est | occupied_gt

    # print('IoU:', intersection.sum() / union.sum())
    
    return

interactive(children=(Dropdown(description='scene', options=('ignatius', 'family'), value='ignatius'), Output(…